In [1]:
from sklearn.preprocessing import LabelBinarizer

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.constraints import maxnorm
from keras import regularizers
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.callbacks import TensorBoard

import numpy as np 
import json
import os
import cv2
import h5py
from time import time

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
from helpers import TrainingMonitor
from helpers import Utils
from helpers import VGG_BASE

In [3]:
output_path = "../output/"

In [4]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train, axis=0)
x_train -= mean
x_test -= mean

lb = LabelBinarizer()
y_train = lb.fit_transform(y_train) 
y_test = lb.fit_transform(y_test)

db_train = h5py.File("../input/datasets/cifar_rgbmean_train.hdf5")
db_test = h5py.File("../input/datasets/cifar_rgbmean_test.hdf5")

x_train_rgbmean = db_train["images"][:].astype('float32')
x_test_rgbmean = db_test["images"][:].astype('float32')

mean = np.mean(x_train_rgbmean, axis=0)
x_train_rgbmean -= mean
x_test_rgbmean -= mean

y_train_rgbmean = db_train["labels"][:]
y_test_rgbmean = db_test["labels"][:]

In [7]:
transfer_model = VGG_BASE()
transfer_model.load_weights(output_path + "saved/vgg_base_weight_86.03.hdf5")
transfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 32)        896       
_________________________________________________________________
block1_bn1 (BatchNormalizati (None, 32, 32, 32)        128       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 32)        9248      
_________________________________________________________________
block1_bn2 (BatchNormalizati (None, 32, 32, 32)        128       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 32)        0         
_________________________________________________________________
block1_drop (Dropout)        (None, 16, 16, 32)        0         
__________

In [8]:
base_model = Model(inputs=transfer_model.input, outputs=transfer_model.get_layer('block3_drop').output)

In [9]:
class FC:
    @staticmethod
    def build(base_model, classes): 
        x = base_model.output
        x = Flatten()(x)
        x = Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
        x = x = BatchNormalization(axis=-1)(x)
        x = Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
        x = x = BatchNormalization(axis=-1)(x)
        x = Dense(classes, activation="softmax")(x)
        
        return x

In [10]:
extended_model = FC.build(base_model, 10)
model = Model(inputs=base_model.input, outputs=extended_model)

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 32)        896       
_________________________________________________________________
block1_bn1 (BatchNormalizati (None, 32, 32, 32)        128       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 32)        9248      
_________________________________________________________________
block1_bn2 (BatchNormalizati (None, 32, 32, 32)        128       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 32)        0         
_________________________________________________________________
block1_drop (Dropout)        (None, 16, 16, 32)        0         
__________

In [15]:
model.load_weights(output_path + "saved/vgg_fc_wide_weight_97.32.hdf5")
#for layer in base_model.layers:
#    layer.trainable = False

In [16]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
#filepath=output_path + "progress/fc-weights-{epoch:02d}-{val_acc:.4f}.hdf5"
filepath=output_path + "progress/tansfer-vgg-fc-weights-best.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=5)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

In [20]:
history = model.fit(x_train_rgbmean, y_train_rgbmean,
            batch_size=64,
            epochs=5,
            validation_split=0.33,
            shuffle="batch",
            callbacks=callbacks)

Train on 33500 samples, validate on 16500 samples
Epoch 1/5
33500/33500 [==============================] - 29s 860us/step - loss: 0.0862 - acc: 0.9714 - val_loss: 0.0940 - val_acc: 0.9694
Epoch 2/5
33500/33500 [==============================] - 29s 867us/step - loss: 0.0853 - acc: 0.9720 - val_loss: 0.0878 - val_acc: 0.9717
Epoch 3/5
33500/33500 [==============================] - 28s 848us/step - loss: 0.0853 - acc: 0.9719 - val_loss: 0.0906 - val_acc: 0.9704
Epoch 4/5
33500/33500 [==============================] - 29s 862us/step - loss: 0.0852 - acc: 0.9718 - val_loss: 0.0860 - val_acc: 0.9718
Epoch 5/5
33500/33500 [==============================] - 28s 846us/step - loss: 0.0856 - acc: 0.9716 - val_loss: 0.0900 - val_acc: 0.9706


In [21]:
scores = model.evaluate(x_test_rgbmean, y_test_rgbmean, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 97.20%; Val: 97.18%; Test: 97.00%


In [ ]:
model_json = model.to_json()
with open(output_path + "saved/vgg_fc_wide_model_97.32.json", "w") as json_file:
    json_file.write(model_json)